In [100]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [101]:
from sklearn.metrics import classification_report

https://medium.com/stanford-cs224w/graph-neural-networks-for-knowledge-tracing-ef31fdaa5f00

In [102]:
pd.set_option('display.max_columns', None)

In [103]:
df=pd.read_pickle('appmath.pkl')  

In [104]:
df2=pd.read_csv('prereq_edges.csv')

In [105]:
G=nx.Graph()

for _,i in df2.iterrows():    
       
        G.add_edge(i['source_lo_title'],i['dest_lo_title'])
        
subgraphs=[G.subgraph(i) for i in nx.connected_components(G)]

for num,sub in enumerate(subgraphs):
    
    nx.set_node_attributes(sub,num,'subGraphId')

nodeSubGraphId={}
for g in subgraphs:
    for node in g.nodes(data=True):
         nodeSubGraphId[node[0]]= node[1]['subGraphId']
            
subgraphsNodeCountDict={ num: g.number_of_nodes() for num,g in  enumerate(subgraphs)}

In [106]:
for col in df.columns:
    if df[col].dtype.name=='category':
        df[col]=df[col].astype(str)

In [107]:
df['loGraphID']=df.learning_objective_name.apply(lambda x: nodeSubGraphId.get(x) )

In [108]:
difficulty=df[~df.correct.isna()].groupby('atom_id')['correct'].mean().reset_index().rename(columns={'correct':'difficulty'})

In [109]:
difficulty

,atom_id,difficulty
0,000361fc-3126-4f25-93da-3ee3b66021c4,0.571429
1,000785e5-46df-4d0b-a12d-2283db4b8c7f,1.000000
2,000da431-9e35-4309-a3ae-cca9225b8d54,0.638554
3,00170926-4ba4-4d1e-bd86-0ac94107acb1,0.700000
4,001cfd36-68fa-4d74-b077-769f3bf2e0ad,0.712919
...,...,...
16441,fff0e6fc-e78f-41da-bcc8-88a6dceb6cc3,0.581481
16442,fff4801c-d828-453e-b1d4-64369f7b1048,0.666667
16443,fff652bc-b348-4054-9aab-8e67b98b022a,0.714286
16444,fffbe876-95cc-4dd0-9097-c657f446df6a,0.714286


In [110]:
df=df.merge(difficulty,how='left',on='atom_id')

In [111]:
loIndex={lo:i for i,lo in enumerate( df.learning_objective_name.unique())}
df['loIndex']=df['learning_objective_name'].apply(lambda x: loIndex[x])


df2['source_index']=df2.source_lo_title.map(loIndex)
df2['dest_index']=df2.dest_lo_title.map(loIndex)

#df2=df2[['source_index','dest_index']].reset_index(drop=True)
#loEdgeIndex=loEdgeMapping.values.transpose()

In [112]:
#df2[(df2.source_lo_title.isin( df.learning_objective_name.unique()[:5])) | (df2.dest_lo_id.isin( df.learning_objective_name.unique()[:5]))]

In [113]:
#loEdgeMapping=df2[['source_index','dest_index']].reset_index(drop=True)
#loEdgeMapping=loEdgeMapping.dropna(axis=0)
#loEdgeIndex=loEdgeMapping.values.transpose()

In [114]:
#loEdgeIndex

In [115]:
#atomIndex={atomId:num for num,atomId in enumerate( df.atom_id.to_list())}

In [116]:
#df['atomIndex']=df.atom_id.apply(lambda x : atomIndex[x] )

In [117]:
def atomClassifier(x):
    if x==None:
        return 'learningMaterial'
    else:
        return 'question'

In [118]:
df['atom_type']=df.correct.apply(lambda x:atomClassifier(x) )

In [119]:
df['correctBinary']=df.correct*1
df.atom_id=df.atom_id.astype(str)


In [157]:
def user_dataset(df,df2,user):
    
    '''
    df - df should be filtered by goal id
    
    edges
    -------
    lo -> atom
    atom -> atom
    lo -> learning materials (lm)
    atom -> lm
    lm -> atom
    lm -> lm
    
    node / features
    ---------------
    
    lo - radnom number
    atom - dificulty , number of attempt
    lm - time spend 
        
    '''    
    loIndex={lo:i for i,lo in enumerate( df.learning_objective_name.unique())}
    
    
    df['loIndex']=df['learning_objective_name'].apply(lambda x: loIndex[x])
    
    unique_lo=df2[(df2.source_lo_title.isin( df.learning_objective_name.unique())) | (df2.dest_lo_id.isin( df.learning_objective_name.unique()))]
    
    unique_lo['source_index']=unique_lo.source_lo_title.map(loIndex)
    unique_lo['dest_index']=unique_lo.dest_lo_title.map(loIndex)
    
    loEdgeMapping=unique_lo[['source_index','dest_index']].reset_index(drop=True)
    loEdgeMapping=loEdgeMapping.dropna(axis=0)
    loEdgeIndex=loEdgeMapping.values.transpose()
    
    userDf=df[df.user_id==user]    
    userDf=userDf.sort_values(by='interaction_end_time')
    
    atomIndex={atomId:num for num,atomId in enumerate( userDf[userDf.atom_type=='question'].atom_id.to_list())}
    lmIndex={atomId:num for num,atomId in enumerate( userDf[userDf.atom_type=='learningMaterial'].atom_id.to_list())}
    
    # assisgning questions and learningMaterials index seperatly in sinlge columns 'atom_index'
    userDf.loc[userDf.atom_type=='question','atomIndex']=userDf.loc[userDf.atom_type=='question',:].atom_id.apply(lambda x : atomIndex[x] )
    userDf.loc[userDf.atom_type=='learningMaterial','atomIndex']=userDf.loc[userDf.atom_type=='learningMaterial',:].atom_id.apply(lambda x : lmIndex[x] )
    
    #lo-atom
    loAtomEdgeIndex=userDf[userDf.atom_type=='question'][['loIndex','atomIndex']].values.transpose()
    #lo-lm
    loLmEdgeIndex=userDf[userDf.atom_type=='learningMaterial'][['loIndex','atomIndex']].values.transpose()
    
    userDf['attempt_count']=userDf.groupby(by=['atom_id']).cumcount()+1
    
    userDf['atom_type_shift']=userDf.atom_type.shift(-1)
    userDf['atomIndex_shift']=userDf.atomIndex.shift(-1)
    
    #atom features
    atomFeatures = userDf[userDf.atom_type=='question'][['difficulty','duration_s','attempt_count']].to_numpy()
    #learning materials features
    lmFeatures = userDf[userDf.atom_type=='learningMaterial'][['duration_s']].to_numpy()
    #learning objectives
    loFeatures=np.ones(userDf.loIndex.nunique()).reshape(-1,1)
    #target
    target=userDf[userDf.atom_type=='question']['correctBinary'].astype(int).to_numpy()
    
    userDf_shift= userDf[~userDf.atomIndex_shift.isna()].copy()  
    
    #atom-atom
    atomEdgeIndex=userDf_shift[(userDf_shift.atom_type=='question') & (userDf_shift.atom_type_shift=='question')][['atomIndex','atomIndex_shift']].values.transpose()
    #atom-lm
    atomLmEdgeIndex=userDf_shift[(userDf_shift.atom_type=='question') & (userDf_shift.atom_type_shift=='learningMaterial')][['atomIndex','atomIndex_shift']].values.transpose()
    #lm-atom
    LmAtomEdgeIndex=userDf_shift[(userDf_shift.atom_type=='learningMaterial') & (userDf_shift.atom_type_shift=='question')][['atomIndex','atomIndex_shift']].values.transpose()
    #lm-lm
    LmLmEdgeIndex=userDf_shift[(userDf_shift.atom_type=='learningMaterial') & (userDf_shift.atom_type_shift=='learningMaterial')][['atomIndex','atomIndex_shift']].values.transpose()
    featuresDict={'atom':atomFeatures,'lm':lmFeatures,'lo':loFeatures}
    edgesDict={'lo_atom':loAtomEdgeIndex,'lo_lm':loLmEdgeIndex,'lo':loEdgeIndex,'atom': atomEdgeIndex,'atom_lm':atomLmEdgeIndex,'lm_atom':LmAtomEdgeIndex,'lm_lm':LmLmEdgeIndex}
    
    return featuresDict,edgesDict,target
    
    
    
    
    

In [158]:
dftest=df[(df.loGraphID==3) & (df.user_id=='0012ff2b-871a-45d6-8e33-804f7464fa0a' )].reset_index(drop=True)

In [159]:
features, edges,target =user_dataset(dftest,df2,'0012ff2b-871a-45d6-8e33-804f7464fa0a' )

C:\Users\mpathirana\AppData\Local\Temp\ipykernel_6736\2474290902.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_lo['source_index']=unique_lo.source_lo_title.map(loIndex)
C:\Users\mpathirana\AppData\Local\Temp\ipykernel_6736\2474290902.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_lo['dest_index']=unique_lo.dest_lo_title.map(loIndex)


In [180]:
for key,item in features.items():
    print( '{} : {}'.format(key,item.shape))

atom : (293, 3)
lm : (54, 1)
lo : (27, 1)


In [181]:
for key,item in edges.items():
    print( '{} : {}'.format(key,item.shape))

lo_atom : (2, 293)
lo_lm : (2, 54)
lo : (2, 35)
atom : (2, 244)
atom_lm : (2, 49)
lm_atom : (2, 48)
lm_lm : (2, 5)


In [161]:
from torch_geometric.data import HeteroData
import torch 


In [162]:
import torch_geometric.transforms as T

In [163]:
def generate_data(features,edges,y):  
    data = HeteroData()
    #node features
    data['atom'].x=torch.tensor(features['atom']).float()
    data['lo'].x= torch.tensor(features['lo']).float()
    data['lm'].x= torch.tensor(features['lm']).float()
    
    #edge index
    data['lo','lo_atom','atom'].edge_index=torch.from_numpy(edges['lo_atom']).long()
    data['lo','lo_lo','lo'].edge_index= torch.from_numpy(edges['lo']).long()
    data['atom','atom_atom','atom'].edge_index=torch.from_numpy(edges['atom']).long()
    data['lo','lo_lm','lm'].edge_index=torch.from_numpy(edges['lo_lm']).long()
    data['atom','atom_lm','lm'].edge_index=torch.from_numpy(edges['atom_lm']).long()
    data['lm','atom_lm','atom'].edge_index=torch.from_numpy(edges['lm_atom']).long()
    data['lm','lm_lm','lm'].edge_index=torch.from_numpy(edges['lm_lm']).long()

    data['atom'].y=torch.from_numpy(y).long()


    #split=T.RandomNodeSplit(num_test =5)
    #data=split(data)

    size=y.shape[0]
    num_test_nodes =int(y.shape[0]*.35)
    some_indices = torch.randperm(size)[:num_test_nodes]


    test_mask = torch.zeros(y.shape[0], dtype=torch.bool)
    test_mask[some_indices] = True
    data['atom'].test_mask = test_mask

    train_mask = torch.ones(y.shape[0], dtype=torch.bool)
    train_mask[some_indices] = False
    data['atom'].train_mask = train_mask

    normalize=T.NormalizeFeatures()
    data=normalize(data)
    return data

In [164]:
data=generate_data(features, edges,target)

In [165]:
print('data.has_isolated_nod : {}'.format(data.has_isolated_nodes()))
print('data.has_self_loops   : {}'.format(data.has_self_loops()))
print('data.is_undirected    : {}'.format(data.is_directed()))

data.has_isolated_nod : True
data.has_self_loops   : False
data.is_undirected    : True


In [174]:
import torch_geometric
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv, to_hetero
import  torch.nn as nn
import torch 

In [176]:
class GNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels,requires_grad=False)
        self.relu1=nn.ReLU()
        self.conv2 = SAGEConv((-1, -1), out_channels,requires_grad=False)
        self.relu2=nn.ReLU()
        self.sigmoid=nn.Sigmoid()

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
       
        return x


model = GNN(hidden_channels=64, out_channels=2)
model = to_hetero(model, data.metadata(), aggr='mean')

In [177]:
optimizer=torch.optim.Adam(model.parameters(),lr=.1)
criterion=torch.nn.CrossEntropyLoss()
def train():
    model.train()
    optimizer.zero_grad()
    out=model(data.x_dict, data.edge_index_dict)
    #print(out)
    loss=criterion(out['atom'][data['atom'].train_mask],data['atom'].y[data['atom'].train_mask])
    loss.backward()
    optimizer.step()
    return loss

def test():
    model.eval()
    out=model(data.x_dict,data.edge_index_dict)
    #print(out)
    pred=out['atom'].argmax(dim=1)
    #print(out)
    test_correct=pred[data['atom'].test_mask]==data['atom'].y[data['atom'].test_mask]
    test_acc=int(test_correct.sum())/int(data['atom'].test_mask.sum())

   
    classificationReport=classification_report(data['atom'].y[data['atom'].test_mask],pred[data['atom'].test_mask])
    print(classificationReport)
    return test_acc

for epcoh in range(1,501):
    loss=train()
print('Epoch : {} , accuracy : {}'.format(epcoh,loss))
    
test()

Epoch : 500 , accuracy : 0.6130599975585938
              precision    recall  f1-score   support

           0       0.56      0.45      0.50        51
           1       0.54      0.65      0.59        51

    accuracy                           0.55       102
   macro avg       0.55      0.55      0.54       102
weighted avg       0.55      0.55      0.54       102



0.5490196078431373